In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from recipe import *

In [15]:
recipe = await main(nodes)

Requesting data for Node: https://esgf-data.dkrz.de/esg-search/search and CORDEX.output.EUR-11.MPI-CSC.MPI-M-MPI-ESM-LR.historical.r1i1p1.REMO2009.v1.mon.tas...
{'offset': 0, 'limit': 10, 'type': 'File', 'replica': 'false', 'format': 'application/solr+json', 'latest': 'true', 'project': 'CORDEX', 'product': 'output', 'domain': 'EUR-11', 'institute': 'MPI-CSC', 'driving_model': 'MPI-M-MPI-ESM-LR', 'experiment': 'historical', 'ensemble': 'r1i1p1', 'rcm_name': 'REMO2009', 'time_frequency': 'mon', 'variable': 'tas'}
{'project': 'CORDEX', 'product': 'output', 'domain': 'EUR-11', 'institute': 'MPI-CSC', 'driving_model': 'MPI-M-MPI-ESM-LR', 'experiment': 'historical', 'ensemble': 'r1i1p1', 'rcm_name': 'REMO2009', 'rcm_version': 'v1', 'time_frequency': 'mon', 'variable': 'tas'}
<ClientResponse(https://esgf-data.dkrz.de/esg-search/search?offset=0&limit=10&type=File&replica=false&format=application/solr%2Bjson&latest=true&project=CORDEX&product=output&domain=EUR-11&institute=MPI-CSC&driving_mode

In [17]:
recipe.keys()

dict_keys(['CORDEX.output.EUR-11.MPI-CSC.MPI-M-MPI-ESM-LR.historical.r1i1p1.REMO2009.v1.mon.tas'])

In [18]:
from pangeo_forge_recipes.recipes import setup_logging
setup_logging()

In [20]:
# Prune the recipe
recipe_pruned = recipe['CORDEX.output.EUR-11.MPI-CSC.MPI-M-MPI-ESM-LR.historical.r1i1p1.REMO2009.v1.mon.tas'].copy_pruned()

In [21]:
# Run the recipe
run_function = recipe_pruned.to_function()
%time run_function()

pangeo_forge_recipes.recipes.xarray_zarr - INFO - Caching input 'Index({DimIndex(name='time', index=0, sequence_len=2, operation=<CombineOp.CONCAT: 2>)})'
pangeo_forge_recipes.storage - INFO - Caching file 'http://esgf1.dkrz.de/thredds/fileServer/cordex/cordex/output/EUR-11/MPI-CSC/MPI-M-MPI-ESM-LR/historical/r1i1p1/MPI-CSC-REMO2009/v1/mon/tas/v20160419/tas_EUR-11_MPI-M-MPI-ESM-LR_historical_r1i1p1_MPI-CSC-REMO2009_v1_mon_195002-195012.nc'
pangeo_forge_recipes.storage - WARNING - POSIX filesystems don't allow filenames to exceed 255 bytes in length. Truncating the filename slug for path 'http://esgf1.dkrz.de/thredds/fileServer/cordex/cordex/output/EUR-11/MPI-CSC/MPI-M-MPI-ESM-LR/historical/r1i1p1/MPI-CSC-REMO2009/v1/mon/tas/v20160419/tas_EUR-11_MPI-M-MPI-ESM-LR_historical_r1i1p1_MPI-CSC-REMO2009_v1_mon_195002-195012.nc' to 'dkrz.de_thredds_fileserver_cordex_cordex_output_eur-11_mpi-csc_mpi-m-mpi-esm-lr_historical_r1i1p1_mpi-csc-remo2009_v1_mon_tas_v20160419_tas_eur-11_mpi-m-mpi-esm-lr_

FileNotFoundError: http://esgf1.dkrz.de/thredds/fileServer/cordex/cordex/output/EUR-11/MPI-CSC/MPI-M-MPI-ESM-LR/historical/r1i1p1/MPI-CSC-REMO2009/v1/mon/tas/v20160419/tas_EUR-11_MPI-M-MPI-ESM-LR_historical_r1i1p1_MPI-CSC-REMO2009_v1_mon_195002-195012.nc